In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten

In [17]:
df = pd.read_csv('/content/sample_data/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
df.columns = df.columns.str.strip()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

df['Label'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [18]:
X = df.drop('Label', axis=1)
y = df['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [19]:
model_dnn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model_dnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_dnn.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_split=0.2
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9592 - loss: 0.1116 - val_accuracy: 0.9990 - val_loss: 0.0055
Epoch 2/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9986 - loss: 0.0082 - val_accuracy: 0.9993 - val_loss: 0.0032
Epoch 3/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9988 - loss: 0.0058 - val_accuracy: 0.9993 - val_loss: 0.0023
Epoch 4/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9990 - loss: 0.0041 - val_accuracy: 0.9993 - val_loss: 0.0023
Epoch 5/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9989 - loss: 0.0043 - val_accuracy: 0.9994 - val_loss: 0.0023
Epoch 6/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9992 - loss: 0.0034 - val_accuracy: 0.9994 - val_loss: 0.0020
Epoch 7/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9991 - loss: 0.0039 - val_accuracy: 0.9994 - val_loss: 0.0019
Epoch 8/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9991 - loss: 0.0031 - val_accuracy: 0.

In [20]:
y_pred_dnn = (model_dnn.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred_dnn))

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19419
           1       1.00      1.00      1.00     25724

    accuracy                           1.00     45143
   macro avg       1.00      1.00      1.00     45143
weighted avg       1.00      1.00      1.00     45143



In [22]:

X_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(
    X_cnn, y, test_size=0.2, random_state=42
)

model_cnn = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(32, 3, activation='relu'),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_cnn.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_split=0.2
)


Epoch 1/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.9789 - loss: 0.0895 - val_accuracy: 0.9992 - val_loss: 0.0051
Epoch 2/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.9989 - loss: 0.0061 - val_accuracy: 0.9990 - val_loss: 0.0030
Epoch 3/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.9990 - loss: 0.0044 - val_accuracy: 0.9994 - val_loss: 0.0029
Epoch 4/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.9992 - loss: 0.0043 - val_accuracy: 0.9996 - val_loss: 0.0019
Epoch 5/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 19s 34ms/step - accuracy: 0.9992 - loss: 0.0034 - val_accuracy: 0.9995 - val_loss: 0.0018
Epoch 6/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 19s 32ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 0.9994 - val_loss: 0.0023
Epoch 7/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 19s 34ms/step - accuracy: 0.9993 - loss: 0.0023 - val_accuracy: 0.9994 - val_loss: 0.0022
Epoch 8/20
565/565 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.9993 - loss: 0.0029 - 

In [23]:
y_pred_cnn = (model_cnn.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred_cnn))

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19419
           1       1.00      1.00      1.00     25724

    accuracy                           1.00     45143
   macro avg       1.00      1.00      1.00     45143
weighted avg       1.00      1.00      1.00     45143

